<a href="https://colab.research.google.com/github/judelo/algosto/blob/master/Impact_Environemental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mesure de l'impact environmental du Machine Learning

## Introduction

Dans ce TP, on va tenter de mesurer l'impact environnemental d'outils standards de Machine Learning. On va utiliser pour cela deux outils différents. Le premier est le site  https://www.green-algorithms.org qui permet de mesurer une partie des gaz à effet de serre liés aux calculs numériques. Pour utiliser cet outil, il faut lui fournir plusieurs types de données sur les conditions de calcul (type de CPU, GPU, lieu du serveur etc). On va voir comment trouver ces informations. 

Ensuite, nous utiliserons la librairie python CodeCarbon pour mesurer l'impact de nos calculs directement à l'intérieur de Python. Nous comparerons les valeurs obtenues avec celles données par green-algorithms.





## Green Algorithms

Cette section explique comment collecter les données nécessaires à l'utilisation du site. Comme certaines des données nécessaires sont liées au matériel, nous expliquerons comment obtenir ces informations en fonction du système d'exploitation. N'oubliez pas que Google Colab fonctionne sur des machines virtuelles Linux.


### Type, nombre, modèles et coeurs

Si vous utilisez  Colab, vous pouvez choisir de faire tourner votre code sur CPU, GPU ou TPU dans **Exécution > Modifier le type d'exécution**.

Pour connaître le CPU utilisé, vous pouvez utiliser les commandes suivantes : 

- **Linux**: `!lscpu` 
- **MacOS**: `sysctl -n machdep.cpu.brand_string` 
- **Windows**: `wmic cpu get name, numberofcores` 



In [ ]:
 !lscpu

Nombre et modèles de GPUs: 

- **Linux**: `lshw -C display`
- **MacOS**: browse: **System Settings > General > About > Graphics**
- **Windows**: browse: **Task Manager > Performance > GPU**


**Remarque sur Colab**

Si vous utilisez Colab, la commande Linux ci-dessous ne fonctionne pas, mais comme Colab utilise des GPU Nvidia, vous pouvez utiliser la commande spécifique :`!nvidia-smi -L`. 

In [ ]:
 !nvidia-smi -L

Si le modèle de CPU (ou de GPU) n'est pas répertorié comme une valeur possible dans green-algorithms, vous pouvez utiliser la valeur de TDP (Thermal Design Power) par coeur de votre CPU (ou GPU). 

### Mémoire disponible
La mémoire vive est très consommatrice d'énergie. On peut utiliser l'une des commandes suivantes pour la connaître (attention si la commande retourne une valeur en Ko ou Mo il faut la convertir en Go (1 Go = 1024 Mo = 1048576 Ko)

- **Linux**: `!grep MemTotal /proc/meminfo`.
- **MacOS**: `system_profiler SPHardwareDataType | grep "Memory:"`. 
- **Windows**: `systeminfo | findstr /C:"Total Physical Memory"`. 

In [ ]:
!grep MemTotal /proc/meminfo

### Plateforme utilisée pour les calculs

Il est important de déterminer sur quelle plateforme le calcul tourne : ordinateur local, serveur, centre de calcul, etc. Typiquement dans un centre de calcul ou sur un serveur, il faut tenir compte d'autres dépenses d'énergie (refroidissement, onduleurs, etc) dûes aux infrastructures.  



### Lieu

Le principal facteur affectant l'empreinte carbone est le lieu où se trouve l'ordinateur ou le serveur de calcul, car l'énergie électrique ne provient pas des mêmes sources selon le pays, donc connaître ce lieu est indispensable pour évaluer les émissions de CO2 de notre calcul. 

On peut vérifier le lieu d'éxécution du code avec la commande  `curl ipinfo.io` (sous Linux, Windows et MacOS). Pour Colab, on peut déterminer le centre utilisé avec le [lien suivant](https://cloud.google.com/about/locations?hl=es) (ce lieu change quand vous réinitialisez le notebook).

Remarque : le site green-algorithms ne tient pas compte du bilan électrique du lieu en temps réel, pour cela il faudrait utiliser [ce site](https://app.electricitymaps.com/). Les données utilisées sont issues de [ce fichier](https://github.com/GreenAlgorithms/green-algorithms-tool/blob/master/data/latest/CI_aggregated.csv), qui contient des valeurs moyennes par pays.  

In [4]:
!curl ipinfo.io

{
  "ip": "34.126.167.239",
  "hostname": "239.167.126.34.bc.googleusercontent.com",
  "city": "Singapore",
  "region": "Singapore",
  "country": "SG",
  "loc": "1.2897,103.8501",
  "org": "AS396982 Google LLC",
  "postal": "018989",
  "timezone": "Asia/Singapore",
  "readme": "https://ipinfo.io/missingauth"
}

### Temps de calcul et cores usage factor 

Le *temps réel* est le temps mesuré entre le début et la fin du calcul. C'est la valeur qu'il faudra fournir comme  "Runtime" dans l'outil green-algorithms. 

Le *temps processeur* est la quantité de temps pendant laquelle un coeur (GPU, GPU ou TPU) a été utilisé.  (source: [Wikipedia](https://en.wikipedia.org/wiki/Time_(Unix)))

$$
\text{Process time} = \text{user time} + \text{system time}
$$

Le cores usage factor est le pourcentage de coeurs disponibles utilisés par le calcul 
$$
  \text{Usage factor} = \frac{\text{Process time}}{(\text{Real time * number of cores})}
$$

Nous pouvons mesurer le temps de traitement des CPUs et le temps réel passé par le code grâce à la commande `time` (sur Linux et MacOS). Malheureusement, il n'existe pas de commande similaire pour Windows. Il sera nécessaire d'utiliser des bibliothèques spécifiques comme `psutil` pour Python. Nous allons voir un exemple ci-dessous. 

**Question 1: Donnez des exemples où le temps réel peut être supérieur au temps de traitement processeur, et vice versa.**

#### Exemple pour mesurer le temps réel et le temps processeur.

Nous allons utiliser dans ce TP comme exemple de calcul typique un réseau profond pré-entraîné pour le débruitage d'images, présenté dans [Tassano et al, An Analysis and Implementation of the FFDNet Image Denoising Method, IPOL, 2019](https://www.ipol.im/pub/art/2019/231/?utm_source=doi). 

On commence par télécharger le code et le modèle. On installe aussi tous les paquets requis par le script. Ils sont listés dans `requirements.txt` et peuvent être installés grâce à la ligne commentée ci-dessous. Si vous utilisez Google colab, l'exécution de cette ligne n'est pas nécessaire, puisque presque toutes les exigences sont déjà remplies, mais si vous exécutez localement, décommentez la ligne.


In [5]:
#@title
!wget -c https://www.ipol.im/pub/art/2019/231/ffdnet-pytorch.zip
!unzip ffdnet-pytorch.zip
#!pip install -r requirements.txt
!pip uninstall -y imgaug albumentations
#!pip install scikit-image==0.15.0  # sci-kit image is needed by the IPOL version of FFDnet 

--2022-12-04 22:29:17--  https://www.ipol.im/pub/art/2019/231/ffdnet-pytorch.zip
Resolving www.ipol.im (www.ipol.im)... 92.243.17.137
Connecting to www.ipol.im (www.ipol.im)|92.243.17.137|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  ffdnet-pytorch.zip
replace ffdnet-pytorch/functions.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ffdnet-pytorch/functions.py  
replace ffdnet-pytorch/ffdnet.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ffdnet-pytorch/ffdnet.png  
replace ffdnet-pytorch/prepare_patches.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ffdnet-pytorch/prepare_patches.py  
replace ffdnet-pytorch/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ffdnet-pytorch/.DS_Store  
replace ffdnet-pytorch/models.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ffdnet-pytorch/requirements.txt? [y]es, [n]o, [A]ll, [N]one, [r]en

In [6]:
cd ffdnet-pytorch

/content/ffdnet-pytorch


Nous allons utiliser le script `test_ffdnet_ipol.py` pour débruiter l'image `input.png`. Nous utiliserons la commande `time` pour mesurer le temps utilisé par le script, et pour obtenir le temps de calcul processeur. 

In [7]:
!time python test_ffdnet_ipol.py --input input.png --add_noise True --noise_sigma 50


### Testing FFDNet model ###
> Parameters:
	add_noise: True
	input: input.png
	suffix: 
	noise_sigma: 0.19607843137254902
	dont_save_results: False
	no_gpu: False
	cuda: True


rgb: True
im shape: (518, 774, 3)
Loading model ...


real	0m6.646s
user	0m4.161s
sys	0m2.546s


Le temps total passé par le script correspond à la valeur 'real'. Le temps CPU sera la somme des valeurs 'user' et 'sys'. Maintenant nous allons mesurer les mêmes variables en utilisant les bibliothèques `psutil` et `time`.

In [11]:
import psutil
import time

start = time.time()
user_start = psutil.cpu_times().user
system_start = psutil.cpu_times().system

!python test_ffdnet_ipol.py --input input.png --add_noise True --noise_sigma 50

end = time.time()
user_end = psutil.cpu_times().user
system_end = psutil.cpu_times().system

print("real ", end - start)
print("user ", user_end - user_start)
print("sys ", system_end - system_start)


### Testing FFDNet model ###
> Parameters:
	add_noise: True
	input: input.png
	suffix: 
	noise_sigma: 0.19607843137254902
	dont_save_results: False
	no_gpu: False
	cuda: True


rgb: True
im shape: (518, 774, 3)
Loading model ...

real  6.536521911621094
user  4.279999999999973
sys  2.5900000000000034


#### Temps GPU

Pour mesurer le temps de traitement GPU, il est nécessaire d'utiliser l'outil [NVIDIA Nsight Compute] (https://developer.nvidia.com/nsight-compute). 

Nous devrons additionner le temps de tous les processus hérités du script et qui utilisent le GPU. Pour faciliter la tâche, nous pouvons envoyer les données dans un fichier CSV et y effectuer une somme.

In [12]:
!ncu --csv --metrics gpu__time_active  --target-processes all python test_ffdnet_ipol.py --input input.png --add_noise True --noise_sigma 50 >> gpu.csv

Malheureusement, il n'y a pas d'outil utilisable en ligne de commande qui nous donne à la fois le temps total du script, le temps CPU et le temps GPU.

Nous pourrions ajouter dans le script `test_ffdnet_ipol.py` quelques lignes pour calculer le *temps réel*, comme montré dans l'exemple précédent. 
Pour simplifier (afin de ne pas modifier le script), nous prendrons le temps utilisé par le script comme celui indiqué dans l'exemple ci-dessus. 

### Power Usage Efficiency (PUE)

Le PUE est le coefficient d'efficacité du centre de données. Si le PUE n'est pas donné, nous considérons la valeur moyenne donnée en 2020 pour les serveurs : 1.58.
Pour les ordinateurs personnels, nous considérons PUE=1.

Pour les centres de données de Google, la valeur est de 1.1 (voir https://www.google.com/about/datacenters/efficiency/).

### PSF (Pragmatic Scaling Factor)

Ce paramètre est utilisé pour indiquer combien de fois nous avons exécuté le code avec la configuration indiquée.

## CodeCarbon

[CodeCarbon] (https://codecarbon.io) est une librairie Python. Elle permet d'estimer la quantité d'émissions CO2 produites par l'exécution d'un code. CodeCarbon prend en compte la consommation d'énergie et la localisation du calcul pour calculer cette empreinte.

### Installation


In [13]:
!pip install codecarbon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 18.5 MB/s 
     |████████████████████████████████| 66 kB 5.1 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 


### Usage

Voir la [documentation](https://mlco2.github.io/codecarbon/usage.html) sur les différentes manières d'utiliser la librairie.

Par défaut, le paquet enregistre les données dans un fichier CSV nommé `emissions.csv` dans le répertoire courant. 

In [14]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()

tracker.start()
!python test_ffdnet_ipol.py --input input.png --add_noise True --noise_sigma 50
emissions: float = tracker.stop()

print("Emissions (KG CO2): ", emissions)

[codecarbon INFO @ 21:41:34] [setup] RAM Tracking...
[codecarbon INFO @ 21:41:34] [setup] GPU Tracking...
[codecarbon INFO @ 21:41:34] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:41:34] [setup] CPU Tracking...
[codecarbon WARNING @ 21:41:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:41:36] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 21:41:36] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 21:41:36] >>> Tracker's metadata:
[codecarbon INFO @ 21:41:36]   Platform system: Linux-5.10.133+-x86_64-with-glibc2.27
[codecarbon INFO @ 21:41:36]   Python version: 3.8.15
[codecarbon INFO @ 21:41:36]   Available RAM : 12.681 GB
[codecarbon INFO @ 21:41:36]   CPU count: 2
[codecarbon INFO @ 21:41:36]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 21:41:36]   GPU count: 1
[codecarbon INFO @ 21:41:36]   GPU model: 1 x Te


### Testing FFDNet model ###
> Parameters:
	add_noise: True
	input: input.png
	suffix: 
	noise_sigma: 0.19607843137254902
	dont_save_results: False
	no_gpu: False
	cuda: True


rgb: True
im shape: (518, 774, 3)
Loading model ...



[codecarbon INFO @ 21:41:42] Energy consumed for RAM : 0.000008 kWh. RAM Power : 4.755459308624268 W
[codecarbon INFO @ 21:41:42] Energy consumed for all GPUs : 0.000052 kWh. All GPUs Power : 30.761 W
[codecarbon INFO @ 21:41:42] Energy consumed for all CPUs : 0.000073 kWh. All CPUs Power : 42.5 W
[codecarbon INFO @ 21:41:42] 0.000133 kWh of electricity used since the begining.


Emissions (KG CO2):  8.407225689846512e-05


Des informations sur l'infrastructure de la plateforme utilisée sont fournies en sortie. On y trouve également l'énergie consommée par les composants et les émissions calculées.

### Visualisation

Pour afficher les informations enregistrées par le tracker sur un tableau de bord, carbonboard crée un site web local sur le port 8050 (http://127.0.0.1:8050). Avec Colab, nous ne pourrons malheureusement pas accéder à cette page. 

Une option consiste à installer le tableau de bord sur l'ordinateur local, et à utiliser le fichier cvs créé dans Colab. Pour ce faire, il faut suivre les étapes suivantes :

- Installer tous les paquets nécessaires pour le tableau de bord

  ```
  pip install codecarbon
  pip install dash
  pip install dash_bootstrap_components==0.13.1
  pip install fire
  ```
- Téléchargez le fichier `emissions.csv` créé dans Colab.
- Exécutez `carbonboard --filepath="/<location>/<of>/emissions.csv"`.
- Allez le site : http://127.0.0.1:8050



### Attention 

Le suivi de l'utilisation CPU et GPU nécessite un logiciel spécifique pour chaque système d'exploitation, défini dans https://pypi.org/project/codecarbon/#cpu (rubrique *Support de l'infrastructure*).

Dans le cas où le logiciel n'est pas installé, ou si l'accès aux fichiers requis ne peut être activé, une consommation moyenne est utilisée, également détaillée [ici](https://pypi.org/project/codecarbon/#cpu). Cette hypothèse peut conduire à estimer des valeurs d'émissions de carbone non réalistes.

## EXERCICE A FAIRE 

Utilisez l'outil https://www.green-algorithms.org et la bibliothèque CodeCarbon pour calculer l'empreinte carbone correspondant à l'exécution dans Colab du script de débruitage `test_ffdnet_ipol.py ` sur `input.png`. L'outil Green-Algorithms ne permet d'utiliser que des temps d'exécution supérieurs à 1 minute, et le script ffdnet prend seulement quelques secondes pour débruiter une image.
On va donc lancer le script 15 fois de suite, en utilisant la ligne suivante :

` ! for i in {0..14} ; do python test_ffdnet_ipol.py --input input.png --add_noise True --noise_sigma 50 ; done`


In [ ]:
TO DO

[codecarbon INFO @ 18:25:32] [setup] RAM Tracking...
[codecarbon INFO @ 18:25:32] [setup] GPU Tracking...
[codecarbon INFO @ 18:25:32] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:25:32] [setup] CPU Tracking...
[codecarbon WARNING @ 18:25:32] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:25:33] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:25:33] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 18:25:33] >>> Tracker's metadata:
[codecarbon INFO @ 18:25:33]   Platform system: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
[codecarbon INFO @ 18:25:33]   Python version: 3.7.15
[codecarbon INFO @ 18:25:33]   Available RAM : 12.681 GB
[codecarbon INFO @ 18:25:33]   CPU count: 2
[codecarbon INFO @ 18:25:33]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 18:25:33]   GPU count: 1
[codecarbon INFO @ 18:25:33]   GPU mod

Emissions (KG CO2):  1.8148471654849708e-05


**Question 2** : comparer les résultats fournis par CodeCarbon avec ceux obtenus par Green-Algorithms.

Pouvez-vous expliquer les différences ? 

Il semble que CodeCarbon n'ait pas détecté que le code s'exécute sur le cloud (vérifiez-le dans le fichier `emissions.csv`), et également qu'il n'ait pas tenu compte du CPU par Colab (il a pris un TDP par défaut), d'après les messages d'erreur suivants :

```
[codecarbon WARNING @ 18:34:09] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:34:10] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
```




**Question 3**: Comment tenir compte du fait qu'on utilise un serveur dans un centre de données pour modifier le résultat de Code Carbon ?


**Question 4**: A votre avis, quels autres facteurs devraient être ajoutés à ce calcul d'impact ?